## Load images

In [ ]:
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [ ]:
vehicles = []
non_vehicles = []

for file in glob.glob( 'train_data/vehicles/**/*.png'):
    vehicles.append( file )

for file in glob.glob( 'train_data/non-vehicles/**/*.png'):
    non_vehicles.append( file )

## Extract Features

The features should be extracted for the following images

In [ ]:
%matplotlib inline
import matplotlib.image as mpimg

car_ex = mpimg.imread(vehicles[0])
noncar_ex = mpimg.imread(non_vehicles[0])

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,5))
ax1.imshow(car_ex)
ax1.set_title('Car Image', fontsize=30)
ax2.imshow(noncar_ex)
ax2.set_title('Noncar Image', fontsize=30)

While Parameter tuning found the following paramters for optimal accurency:

In [ ]:
from feature_extraction import extract_features, get_hog_features

image = mpimg.imread(vehicles[2])
image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)

orient = 9
pix_per_cell = 8
cell_per_block = 2
spatial_size = (32, 32)
hog_channel='ALL'
nb_bins = 32

channel = image[:,:,0]
features, img = get_hog_features( channel, orient, pix_per_cell, cell_per_block, True, True )

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,5))
ax1.imshow(channel)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(img)
ax2.set_title('Gradient', fontsize=30)
ax3.plot( features )
ax3.set_title('Histogram', fontsize=30)

Do the following for all images

In [ ]:
params = dict(
    cspace='YCrCb',
    orient=orient,
    pix_per_cell=pix_per_cell,
    cell_per_block=cell_per_block,
    hog_channel=hog_channel, 
    nb_bins=nb_bins,
    spatial_size=spatial_size
)

vehicle_features, non_vehicle_features = extract_features( vehicles, **params ), extract_features( non_vehicles, **params )

## Train model

In [ ]:
rand_state = np.random.randint(0, 100)

X = np.vstack((vehicle_features, non_vehicle_features)).astype(np.float64)
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

y = np.hstack((np.ones(len(vehicle_features)), np.zeros(len(non_vehicle_features)))) 
X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=rand_state)

In [ ]:
svc = LinearSVC(C=0.0001)
svc.fit(X_train, y_train)
svc.score(X_test, y_test)

In [ ]:
from sklearn.externals import joblib

params['scaler'] = X_scaler
params['svc'] = svc

joblib.dump( params, 'model.pkl')